## Some Data Fetch Graph and Forecast Examples.
The interesting thing here is that we are going to automaet forecasting using Prophet and a quick graph.
Install these if needed: yfinance prophet streamlit plotly

You could also create a dedicated environment...
In Bash:

    conda create -n econdash python=3.10
    conda activate econdash
    pip install pandas yfinance prophet plotly streamlit ipykernel
    python -m ipykernel install --user --name=econdash --display-name "Python (econdash)"


In [33]:
pip install yfinance prophet plotly streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
import plotly.express as px
import streamlit as st
import plotly.graph_objects as go

### Fetch the data!
This arrives with Close, High, Low, Open Prices, Volume and a Date index composed as yyyy-mm-dd
When monthly parameter, we have first day of the month. We can choose also a daily feed.

In [35]:
df = yf.download('^GSPC', period='5y', interval='1mo')
df.head(3)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2020-06-01,3100.290039,3233.129883,2965.659912,3038.780029,131458880000
2020-07-01,3271.120117,3279.989990,3101.169922,3105.919922,96928130000
2020-08-01,3500.310059,3514.770020,3284.530029,3288.260010,82466520000


In [36]:
df=df.reset_index()
df.head(3)

Price,Date,Close,High,Low,Open,Volume
Ticker,,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
0,2020-06-01,3100.290039,3233.129883,2965.659912,3038.780029,131458880000
1,2020-07-01,3271.120117,3279.989990,3101.169922,3105.919922,96928130000
2,2020-08-01,3500.310059,3514.770020,3284.530029,3288.260010,82466520000


We added a column for the index, now, we flatten the MultiIndex to meet the forecasting tool requirements: just one row for the columnds.

In [37]:
if isinstance(df.columns, pd.MultiIndex):
    df.columns = ['_'.join(col).strip() if isinstance(
        col, tuple) else col for col in df.columns]
df.head(3)

,Date_,Close_^GSPC,High_^GSPC,Low_^GSPC,Open_^GSPC,Volume_^GSPC
0,2020-06-01,3100.290039,3233.129883,2965.659912,3038.780029,131458880000
1,2020-07-01,3271.120117,3279.989990,3101.169922,3105.919922,96928130000
2,2020-08-01,3500.310059,3514.770020,3284.530029,3288.260010,82466520000


Then, rename columns:

In [38]:
df = df.rename(
    columns={col: 'y' for col in df.columns if 'close' in col.lower()})
df = df.rename(
    columns={col: 'ds' for col in df.columns if 'date' in col.lower()})
df.head(3)

,ds,y,High_^GSPC,Low_^GSPC,Open_^GSPC,Volume_^GSPC
0,2020-06-01,3100.290039,3233.129883,2965.659912,3038.780029,131458880000
1,2020-07-01,3271.120117,3279.989990,3101.169922,3105.919922,96928130000
2,2020-08-01,3500.310059,3514.770020,3284.530029,3288.260010,82466520000


Now we drop columns and clean.

In [39]:
df = df[['ds', 'y']].dropna()
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df = df.dropna()
df.head(3)

,ds,y
0,2020-06-01,3100.290039
1,2020-07-01,3271.120117
2,2020-08-01,3500.310059


Forecast using Prophet.

In [40]:
model = Prophet()
model.fit(df)
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

01:26:17 - cmdstanpy - INFO - Chain [1] start processing
01:26:17 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\filos\AppData\Local\Programs\Python\Python311\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [41]:
forecast_plot = pd.merge(forecast[['ds', 'yhat']], df[['ds', 'y']], on='ds', how='left')
cutoff_date = df['ds'].max()

# Historical
historical = forecast[forecast['ds'] <= cutoff_date]
# Forecast (future only)
future_forecast = forecast[forecast['ds'] > cutoff_date]

Create the visuals

In [64]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=historical['ds'],
    y=df['y'],
    mode='lines',
    name='Observed',
    line=dict(color='grey')
))

# Forecast line
fig.add_trace(go.Scatter(
    x=future_forecast['ds'],
    y=future_forecast['yhat'],
    mode='lines',
    name='Forecast',
    line=dict(color='#005EB8')  # Pantone 300
))

# Uncertainty area (shaded region between yhat_lower and yhat_upper)
fig.add_trace(go.Scatter(
    x=list(future_forecast['ds']) + list(future_forecast['ds'])[::-1],
    y=list(future_forecast['yhat_upper']) + list(future_forecast['yhat_lower'])[::-1],
    fill='toself',
    fillcolor='rgba(0, 94, 184, 0.2)',  # Pantone blue with transparency
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='95% Confidence Interval'
))

# Layout settings
fig.update_layout(
    title=dict(text='S&P 500 Forecast (Next 12 Months)',
    font=dict(size=18, family='Arial', color='black')
    ),
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis=dict(showline=True, linewidth=1, linecolor='black'),
    yaxis=dict(showline=True, linewidth=1, linecolor='black'),
    annotations=[
        dict(
            text='Note: GSPC, 12mo forecast. SGMS 2025.',
            xref='paper', yref='paper',
            x=1, y=0,  # bottom-right
            showarrow=False,
            font=dict(size=11, color='dark grey'),
            xanchor='right', yanchor='bottom'
        )
    ],
    legend=dict(
        x=0.01,
        y=0.99,
    )
)

fig.show()

Display in Streamlit